## Weaviate Query Agent with Azure OpenAI

This notebook will show you how to define the Weaviate Query Agent as a tool through Azure OpenAI.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
2. Have an Azure project org and `azure_endpoint` and API key. More details can be found [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/overview).
3. Install the OpenAI package and import the `AzureOpenAI` library
4. Install the Weaviate Agents package with `pip install weaviate-agents`
5. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.

### Azure OpenAI Resources
[Learn more](https://learn.microsoft.com/en-us/azure/ai-foundry/what-is-ai-foundry) about the Azure AI Foundry platform and how you can build generative AI applications with it!


### Import libraries and keys

In [1]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

from openai import AzureOpenAI

In [2]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""

### Initialize the Azure OpenAI client

In [9]:
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview"
)

deployment_name = "query-agent-gpt-4" # define the deployment name you want to use 

### Define Query Agent function

In [ ]:
def query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

### Define conversation flow and tool

In [ ]:
def run_conversation():
    # Initial user message
    messages = [{"role": "user", "content": "How can I run Weaviate on Docker?"}]

    # Define the function for the model
    tools = [
    {
        "type": "function",
        "function": {
        "name": "query_agent_request",
        "description": "Send a query to the database and get the response",
        "parameters": {
            "type": "object",
            "properties": {
            "query": {
                "type": "string",
                "description": "Any question or query to search for in the database"
            }
            },
            "required": ["query"]
        }
        }
    }
    ]

    response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)


    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "query_agent_request":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                weaviate_query = query_agent_request(
                    query=function_args.get("query")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "query_agent_request",
                    "content": weaviate_query,
                })
    else:
        print("No tool calls were made by the model.")  

    
    final_response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    )

    return final_response.choices[0].message.content

print(run_conversation())

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rmAbogLS00maXseTrH3HU3cy', function=Function(arguments='{"query":"run Weaviate on Docker"}', name='query_agent_request'), type='function')])
Function arguments: {'query': 'run Weaviate on Docker'}
To run Weaviate on Docker, you must have Docker installed and configured on your system. Here are the steps to get Weaviate running using Docker:

1. **Pull the Weaviate image from Docker Hub:**
   Start by pulling the official Weaviate image from Docker Hub. You can do this by running the following command in your terminal or command prompt:
   ```bash
   docker pull semitechnologies/weaviate:latest
   ```

2. **Run Weaviate container:**
   Once the image is pulled, you can run a Weaviate container using the following command:
   ```bash
   docker run -d --name weaviate -p 8080:8080 semitechnologies/weaviate:latest
   ```
   